## Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

GPU Ocean-modules:

In [2]:
from gpuocean.ensembles import MultiResOceanStateEnsemble
from gpuocean.utils import IPythonMagic, NetCDFInitialization

In [3]:
%cuda_context_handler gpu_ctx

## Selecting Area

As initial and boundary conditions to the simulation, we use data from the Norkyst800 s-level files:

In [4]:
casename = 'lofoten'

case_info = NetCDFInitialization.getCaseLocation(casename)
case_info.pop("name")

'lofoten'

In [5]:
locals().update(case_info)

## Construct initial conditions

In [6]:
s_source_url = ["/sintef/data/NorKyst800/ocean_his.an.20190716.nc"]

In [7]:
data_args = NetCDFInitialization.getInitialConditions(s_source_url, x0, x1, y0, y1, download_data=False, norkyst_data=False)
data_args.keys()

dict_keys(['sponge_cells', 'NX', 'NY', 'nx', 'ny', 'dx', 'dy', 'g', 'r', 'H', 'eta0', 'hu0', 'hv0', 'angle', 'latitude', 'f', 'boundary_conditions_data', 'boundary_conditions', 'wind_stress', 'wind', 'note', 't0', 'timesteps'])

We assume that the wind and boundary conditions (namely the global textures are equal for all ensembles)!

In [15]:
sim_args = {
    "dt": 0.0,
    "write_netcdf":False,
    "model_time_step": 60,
    "small_scale_perturbation":True,
    "small_scale_perturbation_interpolation_factor":1,
    "small_scale_perturbation_amplitude": 0.00025
     }

A big issue with rescaling is that the model error is constructed with an interpolation factor that has to devide `nx` and `ny`, what in flexible cases is only fulfilled by 1 - thats however a bad correlation scale (basically white noise). 

## Ensemble

In [16]:
sim_argses = [sim_args]*3

levels = [0.5, 1, 2.0]

numParticles_per_level = [20, 10, 5]

In [17]:
from importlib import reload
reload(MultiResOceanStateEnsemble)

multires_ensemble = MultiResOceanStateEnsemble.MulitResOceanStateEnsemble(gpu_ctx, sim_argses, data_args, levels, numParticles_per_level)

This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccura

In [18]:
multires_ensemble.stepToObservation(60)

In [ ]:
for p in multires_ensemble.ensembles[0].particles:
    eta, hu, hv = p.download()